In [ ]:
# 🍉 Watermelon Dataset - Exploratory Data Analysis

수박 오디오 데이터셋의 구조와 특성을 분석하는 노트북입니다.

## 목표
1. 데이터셋 구조 분석
2. 라벨링 규칙 확인 
3. 오디오 파일 형식 및 속성 분석
4. 당도 분포 시각화
5. 오디오 특성 분석


In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
from pathlib import Path
import soundfile as sf
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("📦 라이브러리 로딩 완료!")


In [ ]:
# 데이터셋 경로
data_root = Path("../watermelon_sound_data")

# 수박 폴더 목록 수집
watermelon_folders = [f for f in data_root.iterdir() if f.is_dir()]
watermelon_folders.sort()

print(f"📊 총 수박 개수: {len(watermelon_folders)}개")
print("\n📁 수박 폴더 목록:")
for folder in watermelon_folders:
    print(f"  - {folder.name}")

# 라벨링 규칙 분석
pattern = r"(\d+)_(\d+\.?\d*)"
watermelon_data = []

for folder in watermelon_folders:
    match = re.match(pattern, folder.name)
    if match:
        watermelon_id = int(match.group(1))
        sweetness = float(match.group(2))
        
        # 각 폴더의 하위 구조 분석
        subfolders = {}
        for subfolder in ['audio', 'audios', 'chu', 'picture']:
            subfolder_path = folder / subfolder
            if subfolder_path.exists():
                files = list(subfolder_path.glob('*'))
                subfolders[subfolder] = len([f for f in files if f.is_file()])
            else:
                subfolders[subfolder] = 0
        
        watermelon_data.append({
            'id': watermelon_id,
            'sweetness': sweetness,
            'folder_name': folder.name,
            'folder_path': str(folder),
            **subfolders
        })

# DataFrame으로 변환
df = pd.DataFrame(watermelon_data)
df = df.sort_values('id').reset_index(drop=True)

print(f"\n✅ 성공적으로 파싱된 수박: {len(df)}개")
print(f"🍯 당도 범위: {df['sweetness'].min():.1f} ~ {df['sweetness'].max():.1f}")

df.head(10)


In [ ]:
# 당도 분포 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 히스토그램
axes[0, 0].hist(df['sweetness'], bins=15, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('당도 분포 (히스토그램)', fontweight='bold')
axes[0, 0].set_xlabel('당도 (Brix)')
axes[0, 0].set_ylabel('빈도')
axes[0, 0].grid(True, alpha=0.3)

# 박스 플롯
axes[0, 1].boxplot(df['sweetness'], vert=True)
axes[0, 1].set_title('당도 분포 (박스 플롯)', fontweight='bold')
axes[0, 1].set_ylabel('당도 (Brix)')
axes[0, 1].grid(True, alpha=0.3)

# 산점도 (ID vs 당도)
scatter = axes[1, 0].scatter(df['id'], df['sweetness'], alpha=0.7, s=100, c=df['sweetness'], cmap='viridis')
axes[1, 0].set_title('수박 ID vs 당도', fontweight='bold')
axes[1, 0].set_xlabel('수박 ID')
axes[1, 0].set_ylabel('당도 (Brix)')
axes[1, 0].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[1, 0])

# 바이올린 플롯
parts = axes[1, 1].violinplot([df['sweetness']], positions=[1], showmeans=True)
axes[1, 1].set_title('당도 분포 (바이올린 플롯)', fontweight='bold')
axes[1, 1].set_ylabel('당도 (Brix)')
axes[1, 1].set_xticks([1])
axes[1, 1].set_xticklabels(['당도'])
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 통계 요약
print("📊 당도 통계 요약:")
print(df['sweetness'].describe())
